In [28]:
import numpy as np
import numexpr as ne

粗糙集是波兰理工大学Z.pawlak教授提出用来研究不完整数据，不精确知识的表达、学习，归纳等的一套理论

它是一种新的处理模糊和不确定性问题的数学工具，已被广泛应用于知识发现、机器学习、决策支持、模式识别、专家系统及归纳推理等领域。

粗糙集理论的特点是能够分析隐藏在数据中的事实，又不需要关于数据附加信息。

其主要思想是在保持分类能力不变的前提下，通过知识约简，导出问题的决策或分类规则。

从数学的角度看，粗糙集是研究集合的；从编程的角度看，粗糙集的研究对象是矩阵,只不过是一些特殊的矩阵；从人工智能的角度来看，粗糙集研究的是决策表。


作者：思想永不平凡
链接：https://www.jianshu.com/p/a129b7a6be9e
来源：简书
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/未命名.png">

In [29]:
data = np.array([
        ["病人","头疼","肌肉疼","体温","流感"]
        ,["e1",'是', '是', '正常', '否']
        ,["e2",'是', '是', '高', '是']
        ,["e3",'是', '是', '很高', '是']
        ,["e4",'否', '是', '正常', '否']
        ,["e5",'否', '否', '高', '否']
        ,["e6",'否', '是', '很高', '是']])

In [30]:
data

array([['病人', '头疼', '肌肉疼', '体温', '流感'],
       ['e1', '是', '是', '正常', '否'],
       ['e2', '是', '是', '高', '是'],
       ['e3', '是', '是', '很高', '是'],
       ['e4', '否', '是', '正常', '否'],
       ['e5', '否', '否', '高', '否'],
       ['e6', '否', '是', '很高', '是']], dtype='<U3')

# 決策屬性

In [31]:
def decision_factor(info):
    return info[1:,-1]
decision_factor(data)

array(['否', '是', '是', '否', '否', '是'], dtype='<U3')

In [32]:
def split_decision_factor(info):
    dec_fac = decision_factor(info)
    return {i:((dec_fac==i).dot(np.ones(dec_fac.shape[0])),set(info[1:,0][np.ravel(np.argwhere(dec_fac==i))]),dec_fac==i) for i in np.unique(dec_fac)}
split_decision_factor(data)

{'否': (3.0,
  {'e1', 'e4', 'e5'},
  array([ True, False, False,  True,  True, False])),
 '是': (3.0,
  {'e2', 'e3', 'e6'},
  array([False,  True,  True, False, False,  True]))}

# 论域
### 第一列是“病人”，在这一列，除了第一行，还有六行，即有六个元素或者对象在信息系统中，这就是 U ，是非空有限对象集，称为论域。用集合表示为：
# $$ U=\{e_{1},e_{2},e_{3},e_{4},e_{5},e_{6}\} $$

In [33]:
def universe_of_discourse(info):
    return info[1:,:]

    #{str(j[0]):i for i,j in zip(info[1:,1:],info[1:,:1])}

In [34]:
universe_of_discourse(data)

array([['e1', '是', '是', '正常', '否'],
       ['e2', '是', '是', '高', '是'],
       ['e3', '是', '是', '很高', '是'],
       ['e4', '否', '是', '正常', '否'],
       ['e5', '否', '否', '高', '否'],
       ['e6', '否', '是', '很高', '是']], dtype='<U3')

# 属性 $ (C_{i}) $
### 那什么是属性呢，除了第一列之外的其他列都是属性了
### 属性有条件属性和决策属性，决策属性为最后一列
### 先介绍条件属性：$ (C_{i}) $
### 设$(C_{1})$为头疼属性，为$(C_{2})$肌肉疼属性，为$(C_{3})$体温属性
### 用集合表示为：

### 头疼属性：
# $ C_{1} = \{是,是,是,否,否 \} $  
### 肌肉疼属性：
# $ C_{2} = \{是,是,是,否,是 \} $ 
### 体温属性：
# $ C_{3} = \{正常,高,很高,正常,高,很高 \} $


In [35]:
def attribute_set(info):
    return info[1:,1:-1]

def condition_attribute(info):
    return info[1:,1:-1]

In [36]:
attribute_set(data)

array([['是', '是', '正常'],
       ['是', '是', '高'],
       ['是', '是', '很高'],
       ['否', '是', '正常'],
       ['否', '否', '高'],
       ['否', '是', '很高']], dtype='<U3')

# 流感(D)就是决策属性了
# $ D=\{ 否，是，是，否，否，是  \} $
可以看出，在信息系统中，有很多的对象，它们有着各种各样的属性

其中不同的对象有着不同或者相同的条件属性，使得每个对象有着不同的特征，这些条件属性会影响到决策属性，决策属性不同，将决定着这个信息系统能被分成几类



显然决策属性值的不同直接受条件属性的影响，可以说每一个条件属性都可能成为决定决策属性值的潜在影响因素。进而决定着信息系统中的对象会被划分到哪一类去

但是，我们也知道，影响事物的潜在因素会有很多，有些会显著影响结果，有些会有一些影响，而有些因素却是可有可无的



现在我们正处于大数据的时代，每个人每天的很多行为会产生各种各样的数据

这些数据通常很大，维度很高，里面会有商家感兴趣的部分，但是数据维数太大了

面对这种“数据极其丰富而信息相对缺乏”的情况，如何从海量的数据中获得有用的信息或者决定性因素会是我们很感兴趣的方面

因此，面对这样一个含有大量信息的信息系统，如何去发掘出什么是影响决策属性值的关键属性是很有必要的

而这些关键属性就隐藏在这一列列的条件属性中，有些属性对决策属性产生了重要的影响，而有些属性就显得可有可无了

而我们就是希望能够找出这些对决策属性有着重要影响的条件属性



对于上面的病例，有经验的医生可能一眼就看出来什么是影响一个病人患流感的关键因素

而我们可能作为门外汉，可以学习通过一些粗糙集的知识，同样也能发掘出这样的关键因素！

作者：思想永不平凡
链接：https://www.jianshu.com/p/a129b7a6be9e
来源：简书
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

# 等价类与等价关系

在数学上，假设在一个集合上定义一个等价关系（用来表示），则中的某个元素的等价类就是中等价于的所有元素所形成的子集：
# $$ [a]=\{x\in X|x \sim a   \} $$
粗糙集中的等价类和等价关系

$ S=(U,A=C\cup D,V,f) $ 是决策信息系统，$ \forall B\subseteq C $ ，论域U的不可分辨关系被定义为：

# $$ R_B=\{(x,y)\in U\times U|f(x,a)=f(y,a),\forall a\in B   \} $$
很显然，不可分辨关系是一种等价关系。它将论域U划分为
# $ U/R_{B},U/R_{B}=\{E_{1},E_{2},...,E_{m} \} $ 
是由等价关系 
# $ R_{B} $ 
形成的等价类集合。由等价关系
# $ R_{B} $ 
形成的等价类
# $ [x]_{B}=\{y|(x,y)\in R_{B} \} $
是粗糙集理论中的基本知识粒。





# 设：
## 论域
## $ U=\{e_{1},e_{2},e_{3},e_{4},e_{5},e_{6}  \} $
## 条件属性
## $ c_{i}=\{c_{1},c_{2},c_{3} \} $
## 决策属性
## $ D=\{d  \} $


In [37]:
def getX(info,*keyword):
    return np.array([info[:,np.argwhere(info[0] == key)[0][0]] for key in np.array(keyword)]).T

getX(data,"头疼","肌肉疼")

array([['头疼', '肌肉疼'],
       ['是', '是'],
       ['是', '是'],
       ['是', '是'],
       ['否', '是'],
       ['否', '否'],
       ['否', '是']], dtype='<U3')

# 求出等价商集

In [38]:
def quotient_set(info,*keyword):
    knowledge = getX(info,*keyword)
    univ_disc = universe_of_discourse(knowledge)
    print(f"univ_disc{univ_disc}")
    e = info[1:,0]
    result = {}
    for keys in {tuple(i) for i  in univ_disc}:
        _index = np.ravel(np.argwhere((univ_disc==np.array(keys)).dot(np.ones(len(keyword)))==len(keyword)))
        e_index = e[_index]
        result.update({tuple(e_index):univ_disc[_index]})
    return result

quotient_set(data,"头疼","肌肉疼")

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


{('e1',
  'e2',
  'e3'): array([['是', '是'],
        ['是', '是'],
        ['是', '是']], dtype='<U3'),
 ('e5',): array([['否', '否']], dtype='<U3'),
 ('e4',
  'e6'): array([['否', '是'],
        ['否', '是']], dtype='<U3')}

# 给定关系查询等价类
 ：设R是A上的一个等价关系，与A中一个元素a相关的所有元素的集合被称做a的一个等价类[a]R.当仅考虑一个关系是时，可以省去下标，而简写成[a].形式地，
 $$ [a]R=｛s|(a,s)∈R｝$$
 .



In [39]:
def equivalence_class(C,R,A):
    column = np.ravel([np.argwhere(A[0]==c) for c in C])
    change_U = A.T[column].T
    univ_disc = universe_of_discourse(change_U)
    Rm = len(R)
    arr_one = np.ones(Rm)
    e = A[1:,0]
    return e[(univ_disc==R).dot(arr_one)==Rm]

equivalence_class(C =["头疼","肌肉疼"],R=['是', '否'],A = data)

array([], dtype='<U3')

# 列出所有知识粒度的等价关系对应
$\forall B\subseteq C$

In [40]:
import math
from itertools import product

def combinationL(loop_val): # 多list组合函数
    return np.array([i for i in product(*loop_val)])

def drop_dim(List):
    return [j for i in List for j in i]

def comparefunc(A,B,bi,e,D,cname):
    result = dict()
    m,n = A.shape
    t = -1
    for a in A:
        t+=1
        for b in B:
            bn = b.shape[0]
            if bn < n:
                for i in range(math.ceil(n/bn)):
                    temp1 = (a[i:i+bn]==b).all()  
                    if temp1:
                        index_str = (e[t],i,i+bn)
                        key = "{}{}".format(e[bi[t]],index_str)
                        result1 = {key:{"决策属性":D[bi[t]],"论域":e[bi[t]],"条件属性":a,"等价关系":b,"元素段坐标":index_str}}    
                        result.update(result1)
            else:
                temp2 = (a==b).all()
                if temp2:
                    index_str = (e[t],0,bn)
                    key = "{}{}".format(e[bi[t]],index_str)
                    result2 = {key:{"决策属性":D[bi[t]],"论域":e[bi[t]],"条件属性":a,"等价关系":b,"元素段坐标":index_str}}
                    result.update(result2)
    return result

def creater_equivalence_relation(S,C):
    cname = C
    e = S[1:,0] # 论域
    D = S[1:,-1] # 决策属性
    Ddistion = set(D)
    columns = np.ravel([np.argwhere(S[0]==c) for c in C]) #选择列
    C = S.T[columns].T[1:] #选择的条件属性集
    Cname = S.T[columns].T[0]
    Cm ,Cn = C.shape
    Dbool_index = {d:np.ravel(np.argwhere(D==d)) for d in Ddistion } # 决策属性分类
    Cclass =[]
    for ci,bi in Dbool_index.items():
        Cb = C[bi]
        cdistion = [set(Cb[:,c]) for c in range(len(Cb[0]))]
        for i in range(len(cdistion)):
            ccomb = combinationL(cdistion[:i+1])
            temp_c = comparefunc(C[bi],ccomb,bi,e,D,cname)
            Cclass.append(temp_c)
    return Cclass

In [41]:
def equivalence_relation(S,C,keyword=None):
    D = S[1:,-1] # 决策属性
    def filter_(ds):
        result_e , result_R ,result_D ,result_location = [],[],[],[]
        data = creater_equivalence_relation(ds,C)
        for i in data:
            for ik,iv in i.items():
                e = iv['论域']
                R = iv['等价关系']
                D = iv['决策属性']
                location = iv['元素段坐标']
                result_e.append(e)
                result_R.append(R)
                result_D.append(D)
                result_location.append(C[location[-2]:location[-1]])
        def clear(List):
            bool_func = np.array([np.ravel(np.argwhere(np.array(result_e) == coder)) for coder in set(result_e)])
            eq_  = np.array(List)[bool_func]
            return dict(enumerate(eq_))
        return {'论域':clear(result_e),'等价关系':clear(result_R),'决策属性':clear(result_D),'元素段坐标':clear(result_location)}
    if keyword == "splitD":
        Ddistion = set(D)
        DS = {d:filter_(np.vstack((S[0],S[1:][D==d]))) for d in Ddistion}
        return DS
    elif keyword == None:
        return creater_equivalence_relation(S,C)
equivalence_relation(data,["头疼","肌肉疼",'体温'],keyword="splitD")

/tmp/ipykernel_1907640/4083884369.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  eq_  = np.array(List)[bool_func]


{'否': {'论域': {0: array(['e4', 'e4', 'e4', 'e4'], dtype='<U2'),
   1: array(['e1', 'e1', 'e1', 'e1'], dtype='<U2'),
   2: array(['e5', 'e5', 'e5', 'e5'], dtype='<U2')},
  '等价关系': {0: array([array(['否'], dtype='<U1'), array(['是'], dtype='<U1'),
          array(['否', '是'], dtype='<U1'),
          array(['否', '是', '正常'], dtype='<U2')], dtype=object),
   1: array([array(['是'], dtype='<U1'), array(['是'], dtype='<U1'),
          array(['是', '是'], dtype='<U1'),
          array(['是', '是', '正常'], dtype='<U2')], dtype=object),
   2: array([array(['否'], dtype='<U1'), array(['否'], dtype='<U1'),
          array(['否', '否'], dtype='<U1'),
          array(['否', '否', '高'], dtype='<U2')], dtype=object)},
  '决策属性': {0: array(['否', '否', '否', '否'], dtype='<U1'),
   1: array(['否', '否', '否', '否'], dtype='<U1'),
   2: array(['否', '否', '否', '否'], dtype='<U1')},
  '元素段坐标': {0: array([list(['头疼']), list(['肌肉疼']), list(['头疼', '肌肉疼']),
          list(['头疼', '肌肉疼', '体温'])], dtype=object),
   1: array([list(['头疼']), 

# 不可分辨关系
相差不大的个体被归于同一类，它们的关系就是不可分辨关系。给定一个论域U和U上的一簇等价关系S，若P⊆S，且P≠∅，则∩P（P中所有等价关系的交集）仍然是论域U上的一个等价关系，称为P上的一个不可分辨关系

# 给定X

In [42]:
def definitionX(info,X,*keyword):
    knowledge = getX(info,*keyword)
    univ_disc = universe_of_discourse(knowledge)
    data_dict = {i:j for i,j in zip(info[1:,0],univ_disc)}
    return {X:np.array([data_dict[i] for i in X])} ,univ_disc

# 上/下近似/边界线

In [43]:
from itertools import combinations

def Rouh_set_all(info,X,*keyword,change_approx = 'lower'):
    quot_set = quotient_set(info,*keyword)
    X ,knowledga = definitionX(info,X,*keyword)
    intersect_list,index_list = [],[]
    for set_ in quot_set:
        intersect = (set(set_)&set(list(X)[0]))
        lenght = len(intersect)    
        if lenght > 0:
            intersect_list.append(set_)
            index_list.append(lenght)
    if change_approx == 'lower':
        return set(intersect_list[index_list.index(min(index_list))])
    elif change_approx == "upper":
        return {j for i in range(len(intersect_list)) for j in intersect_list[i]}
    elif change_approx == "boundary_region":
        upper = set([j for i in range(len(intersect_list)) for j in intersect_list[i]])
        lower = set(list(intersect_list[index_list.index(min(index_list))]))
        return upper-lower
    elif change_approx=="positive_field":
        return set(list(intersect_list[index_list.index(min(index_list))]))
    elif change_approx == "negative_field":
        univ_disc = set(info[1:,0])
        return univ_disc-set(intersect_list[index_list.index(min(index_list))])
    elif change_approx == "R_exact_sets":
        temp = [list(combinations(intersect_list,i)) for i in range(1,len(intersect_list))]
        exact = [e for e in drop_dim(temp) if set(len(e)==1 and e[0] or drop_dim(e))==set(X)]
        lenght_set = len(exact)
        return lenght_set > 0 and {"R exact sets":exact} or "Rouh set" 


In [44]:
for i in ['lower',"upper","boundary_region","positive_field","negative_field","R_exact_sets"]:
    print(i,"=",Rouh_set_all(data,("e2","e3","e5"),"头疼","肌肉疼",change_approx=i))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]
lower = {'e5'}
univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]
upper = {'e5', 'e2', 'e1', 'e3'}
univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]
boundary_region = {'e2', 'e1', 'e3'}
univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]
positive_field = {'e5'}
univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]
negative_field = {'e6', 'e4', 'e3', 'e2', 'e1'}
univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]
R_exact_sets = Rouh set


# 下近似

In [45]:
def lower_approximation(info,X,keyword):
    return Rouh_set_all(info,X,*keyword,change_approx = 'lower')

lower_approximation(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


{'e5'}

# 上近似

In [46]:
def upper_approximation(info,X,keyword):
    return Rouh_set_all(info,X,*keyword,change_approx = 'upper')
upper_approximation(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


{'e1', 'e2', 'e3', 'e5'}

# 边界线

In [47]:
def boundary_region(info,X,keyword):
    return Rouh_set_all(info,X,*keyword,change_approx = 'boundary_region')

boundary_region(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


{'e1', 'e2', 'e3'}

# 正域

In [48]:
def positive_field(info,X,keyword):
    return Rouh_set_all(info,X,*keyword,change_approx = 'positive_field')
positive_field(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


{'e5'}

# 负域

In [49]:
def negative_field(info,X,keyword):
    return Rouh_set_all(info,X,*keyword,change_approx = 'negative_field')
negative_field(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


{'e1', 'e2', 'e3', 'e4', 'e6'}

# 精确集

In [50]:
def R_exact_sets(info,X,keyword):
    test = Rouh_set_all(info,X,*keyword,change_approx = 'R_exact_sets')
    return test != 'Rouh set' and test or "this relationship has no R exact set"
R_exact_sets(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


'this relationship has no R exact set'

# 检验给定X是否是论域的粗糙集

In [51]:
def rouh_set(info,X,keyword):
    return Rouh_set_all(info,X,*keyword,change_approx = 'R_exact_sets') == 'Rouh set'
rouh_set(data,("e2","e3","e5"),("头疼","肌肉疼"))

univ_disc[['是' '是']
 ['是' '是']
 ['是' '是']
 ['否' '是']
 ['否' '否']
 ['否' '是']]


True

# 近似分类的:精度/粗糙度/质量

In [52]:
def approximate_classification(info,Uname,changeD):
    U , n = info.shape
    Dinfo = {d:np.vstack((info[0],info[1:][info[1:,-1]==d])) for d in set(info[1:,-1])}
    new_info = Dinfo[changeD]
    e = tuple(new_info[1:,0])
    if rouh_set(info,e,Uname):
        lowerR = lower_approximation(info,e,Uname)
        pperR = upper_approximation(info,e,Uname)
        boundaryR = boundary_region(info,e,Uname)
        positiveR = positive_field(info,e,Uname)
        negativeR = negative_field(info,e,Uname)
        alphaR = len(lowerR)/len(pperR)
        rhoR = 1- alphaR
        gammaR = len(lowerR)/U
        return {"近似分类精度":alphaR,"粗糙度":rhoR,"近似分类质量":gammaR,"上近似":pperR,"下近似":lowerR,"决策边界":boundaryR,"正域":positiveR,"负域":negativeR}
    else:
        test = Rouh_set_all(info,e,*Uname,change_approx = 'R_exact_sets')
        return test
approximate_classification(data,('头疼', '肌肉疼', '体温'),"是")

univ_disc[['是' '是' '正常']
 ['是' '是' '高']
 ['是' '是' '很高']
 ['否' '是' '正常']
 ['否' '否' '高']
 ['否' '是' '很高']]
univ_disc[['是' '是' '正常']
 ['是' '是' '高']
 ['是' '是' '很高']
 ['否' '是' '正常']
 ['否' '否' '高']
 ['否' '是' '很高']]
univ_disc[['是' '是' '正常']
 ['是' '是' '高']
 ['是' '是' '很高']
 ['否' '是' '正常']
 ['否' '否' '高']
 ['否' '是' '很高']]
univ_disc[['是' '是' '正常']
 ['是' '是' '高']
 ['是' '是' '很高']
 ['否' '是' '正常']
 ['否' '否' '高']
 ['否' '是' '很高']]
univ_disc[['是' '是' '正常']
 ['是' '是' '高']
 ['是' '是' '很高']
 ['否' '是' '正常']
 ['否' '否' '高']
 ['否' '是' '很高']]
univ_disc[['是' '是' '正常']
 ['是' '是' '高']
 ['是' '是' '很高']
 ['否' '是' '正常']
 ['否' '否' '高']
 ['否' '是' '很高']]


{'近似分类精度': 0.3333333333333333,
 '粗糙度': 0.6666666666666667,
 '近似分类质量': 0.14285714285714285,
 '上近似': {'e2', 'e3', 'e6'},
 '下近似': {'e6'},
 '决策边界': {'e2', 'e3'},
 '正域': {'e6'},
 '负域': {'e1', 'e2', 'e3', 'e4', 'e5'}}

# 知识约简

In [53]:
from itertools import permutations,combinations

def knowledge_reduction(info,changeD):
    e = info[0][1:-1]
    comb_e = drop_dim([list(combinations(e,i)) for i in range(1,len(e))])
    appro_class = {comb:[comb,approximate_classification(data,comb,changeD)] for comb in comb_e}
    str_appro_class = [str(e) for e in appro_class.values()]
    distion_str_app  = set(str_appro_class)
    counter = np.array([str_appro_class.count(i) for i in distion_str_app])
    return [eval(tuple(distion_str_app)[i]) for i in np.ravel(np.argwhere(counter==counter.min()))]
knowledge_reduction(data,"否")

univ_disc[['是']
 ['是']
 ['是']
 ['否']
 ['否']
 ['否']]
univ_disc[['是']
 ['是']
 ['是']
 ['否']
 ['否']
 ['否']]
univ_disc[['是']
 ['是']
 ['是']
 ['否']
 ['否']
 ['否']]
univ_disc[['是']
 ['是']
 ['是']
 ['否']
 ['否']
 ['否']]
univ_disc[['是']
 ['是']
 ['是']
 ['否']
 ['否']
 ['否']]
univ_disc[['是']
 ['是']
 ['是']
 ['否']
 ['否']
 ['否']]
univ_disc[['是']
 ['是']
 ['是']
 ['是']
 ['否']
 ['是']]
univ_disc[['是']
 ['是']
 ['是']
 ['是']
 ['否']
 ['是']]
univ_disc[['是']
 ['是']
 ['是']
 ['是']
 ['否']
 ['是']]
univ_disc[['是']
 ['是']
 ['是']
 ['是']
 ['否']
 ['是']]
univ_disc[['是']
 ['是']
 ['是']
 ['是']
 ['否']
 ['是']]
univ_disc[['是']
 ['是']
 ['是']
 ['是']
 ['否']
 ['是']]
univ_disc[['正常']
 ['高']
 ['很高']
 ['正常']
 ['高']
 ['很高']]
univ_disc[['正常']
 ['高']
 ['很高']
 ['正常']
 ['高']
 ['很高']]
univ_disc[['正常']
 ['高']
 ['很高']
 ['正常']
 ['高']
 ['很高']]
univ_disc[['正常']
 ['高']
 ['很高']
 ['正常']
 ['高']
 ['很高']]
univ_disc[['正常']
 ['高']
 ['很高']
 ['正常']
 ['高']
 ['很高']]
univ_disc[['正常']
 ['高']
 ['很高']
 ['正常']
 ['高']
 ['很高']]
univ_disc[['是' '是']
 ['是' '是']
 ['是' '是'

[[('头疼',),
  {'近似分类精度': 0.5,
   '粗糙度': 0.5,
   '近似分类质量': 0.42857142857142855,
   '上近似': {'e1', 'e2', 'e3', 'e4', 'e5', 'e6'},
   '下近似': {'e1', 'e2', 'e3'},
   '决策边界': {'e4', 'e5', 'e6'},
   '正域': {'e1', 'e2', 'e3'},
   '负域': {'e4', 'e5', 'e6'}}],
 [('体温',),
  {'近似分类精度': 0.5,
   '粗糙度': 0.5,
   '近似分类质量': 0.2857142857142857,
   '上近似': {'e1', 'e2', 'e4', 'e5'},
   '下近似': {'e2', 'e5'},
   '决策边界': {'e1', 'e4'},
   '正域': {'e2', 'e5'},
   '负域': {'e1', 'e3', 'e4', 'e6'}}],
 [('肌肉疼',),
  {'近似分类精度': 0.16666666666666666,
   '粗糙度': 0.8333333333333334,
   '近似分类质量': 0.14285714285714285,
   '上近似': {'e1', 'e2', 'e3', 'e4', 'e5', 'e6'},
   '下近似': {'e5'},
   '决策边界': {'e1', 'e2', 'e3', 'e4', 'e6'},
   '正域': {'e5'},
   '负域': {'e1', 'e2', 'e3', 'e4', 'e6'}}],
 [('头疼', '肌肉疼'),
  {'近似分类精度': 0.5,
   '粗糙度': 0.5,
   '近似分类质量': 0.42857142857142855,
   '上近似': {'e1', 'e2', 'e3', 'e4', 'e5', 'e6'},
   '下近似': {'e1', 'e2', 'e3'},
   '决策边界': {'e4', 'e5', 'e6'},
   '正域': {'e1', 'e2', 'e3'},
   '负域': {'e4', 'e5', 'e6'}}],